# packges

In [21]:

import pandas as pd
import requests
import os

In [22]:
!pip install SybilChecks




Defaulting to user installation because normal site-packages is not writeable


In [23]:
from SybilChecks import checks as c


# importing data

In [24]:
df = pd.read_excel('gr15_grants.xlsx')

In [26]:
df.shape

(1503, 16)

In [27]:
df.columns

Index(['grant_id', 'active', 'title', 'address', 'amount_received',
       'amount_received_in_round', 'contribution_count', 'contributor_count',
       'description', 'website', 'github_project_url', 'twitter_handle_2',
       'twitter_handle_1', 'twitter_verified', 'created_on', 'last_update'],
      dtype='object')

In [28]:
sample  = df[0:10] 

# Legos

In [29]:
# Legos validations 

# website Lego 
 # Todo :
 # needs to work with redirection codes 300s 
    # right now it classifies redirections as 'False' (not working), it should be able to get the redirect URL and validates it

def website_validation(website):
    website_working = []
    website_status = requests.head(website,timeout = 4, verify = False).status_code
    if (website_status == 200):
        website_working.append(True)
    else:
        website_working.append(False)

    return website_working[0]

# appying legos

### website validation Lego

In [ ]:


website_working = []
for i in sample['website']:
    website_working.append(website_validation(i))

### wallet validation Lego 

In [ ]:
# validates if wallet have been initialized ( made it first transaction during specified period)
round_start = '2023-01-17'
round_finish = '2023-01-31'
api_key = '' #covalent API key 
chain_id = '1' #Etherium chain code
first_trx_d_round = []

for i in sample['address']:
    x = c.first_trx_during_round(i, api_key, chain_id, round_start, round_finish)
    first_trx_d_round.append(x[1])



In [ ]:
# there is other validations on the SybilChecks package, it could study to understand its applications here 

# building results dataframe


In [35]:

result_df =  pd.DataFrame( data = {'wallets' :sample['address'] ,
                                'first_trx_d_round_lego' : first_trx_d_round, 
                                'website_working_lego' : website_working})
result_df


,wallets,first_trx_d_round,website_working
0,0xE7048aB8dFA7F7a80d79fD6c5AcF8B0C1d174fdF,False,True
1,0xcC17a95B3f5861BC2cfD28ca81d611077ef36ED1,False,True
2,0x715465a41a34Fb1Af53328708171c0FBf2B88825,False,True
3,0x7ABDd18D37571e1BA884A8fA9E07a3612b584661,False,False
4,0xF77494B20F2DF03a7663465a001486F834739A78,False,True
5,0x03f142529a7B70305C07a50fAA44f6EBDADB4624,False,False
6,0x68895B58d148032d74322CcCE9C8740431435E0B,False,False
7,0xfaD124800817bE2dC09BF3a9c77BF7BE9F280626,False,True
8,0x226dAaeBc9d864533f8F86493225F2E3B91565A8,False,False
9,0xdF576d166aF1f45e4E42D1eF872b512551b6d102,False,False


In [1]:
import os
import pandas as pd
os.chdir('../src/') #to avoid module load issues. TODO fix via module path

from src.lib.utils.twitter import get_user_info

def get_grant_prj_twitter_info(x):

    twitter_account = get_user_info(x['twitter_handle_1'])
    
    #num_tweets, created_at, followers_count
    return x['grant_id'], x['twitter_handle_1'], twitter_account.statuses_count, twitter_account.created_at, twitter_account.followers_count
#LOAD GRANTS
grant_json_path = 'gr15_grants.json'
df_grants = pd.read_json(grant_json_path).transpose()
#GET GRANT TWITTER INFO
grant_prj_twitter_info = df_grants.head(5).apply(lambda x: get_grant_prj_twitter_info(x), axis=1)
df_grant_prj_twitter_info = pd.DataFrame(grant_prj_twitter_info.tolist())
df_grant_prj_twitter_info.columns = ['grant_id', 'twitter_name', 'num_tweets', 'created_at', 'followers_count']
df_grant_prj_twitter_info.head()

FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: '../src/'